# Storytelling Data Visualization Lab

In this lab you'll use a dataset called `housing_prices.csv` which contains the sales data of houses. The dataset and descriptions of the columns are available from [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). For your convenience, you can review the descriptions of the data columns from [here](https://drive.google.com/file/d/1-iXooLjNuEXU41dqz8ORQ5JEZPHd9x0X/view?usp=sharing).

Pretend you are a data analyst at an investment company where the board decided to make investments in real estates. Your boss asked you to analyze this housing sales dataset and present to the investment managers on **what features of houses are strong indicators of the final sale price**. You need to present your findings in intuitive ways so that the investment managers understand where your conclusions come from.

#### You will use the appropriate data visualization graphs to tell your stories.

## Challenge 1 - Understanding the Dataset

After receiving the data and clarifying your objectives with your boss, you will first try to understand the dataset. This allows you to decide how you will start your research in the next step.

The dataset is [here](https://drive.google.com/file/d/1MRhRtdX8QuPPEhelBIS_FEl5vJjRLSeE/view?usp=sharing). Please download it and place it in the data folder.<br>
First, import the basic libraries and the dataset.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

df = pd.read_csv('../data/housing_prices.csv')

#### As a routine before analyzing a dataset, print the first few rows of the dataset

In [ ]:
df.head()

You find the dataset has 81 columns which are a lot. 

#### Since the column `Id` is meaningless in our data visualization work, let's drop it

In [ ]:
# your code here
df.drop('Id', axis=1, inplace=True)

You care about missing values. If a column has too many missing values, it is not reliable to use it to predict sales price.

#### In the cell below, calculate the percentage of missing values for each column. 

Make a table containing the column name and the percentage of missing values. Print the columns where more than 20% of values are missing. An example of what your output  should look like is [here](https://drive.google.com/file/d/1cuq6qhFZC5wavm-_STcxktBKdAc4xvH8/view?usp=sharing)

[This reference](https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset) can help you make the missing values table.

In [ ]:
# your code here
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

#### Drop the columns you find that have more than 20% missing values.

After dropping, check the shape of your dataframes. You should have 75 columns now.

In [ ]:
# your code here
# 
df.drop(missing_value_df[missing_value_df['percent_missing'] > 20].column_name.tolist(), axis=1, inplace=True)

Since you're asked to analyze sale prices, first let's see if the sale prices (column `SalePrice`) has a normal distribution. This is important because normally distributed data can be better represented with mathematical models.

#### In the cell below, use the propriate graph to visualize the shape of distribution of the sale prices. Then explain what you find from the graph about data distribution.

In [ ]:
# your code here
df['SalePrice'].hist(bins=50)

In [ ]:
# your comment here
#It is definitely not normal, the values are more distributed towards the right of the mean and median,
#giving us a high Kurtosis

## Bonus Challenge 1 - Adjust Data Distribution

If you used the correct method in the previous step, you should have found the data distribution is skewed to the left. In order to improve your data visualization in the next steps, you can opt to adjust the `SalePrice` column by applying a mathematical function to the values. The goal is to produce a bell-shape normal distribution after applying the mathematical function to the sale price.

*This technique is optional in data visualization but you'll find it useful in your future machine learning analysis.*

#### In the cell below, adjust the `SalePrice` column so that the data are normally distributed.

Try applying various mathematical functions such as square root, power, and log to the `SalePrice` column. Visualize the distribution of the adjusted data until you find a function that makes the data normally distributed. **Create a new column called `SalePriceAdjusted` to store the adjusted sale price.**

[This reference](https://trainingdatascience.com/workshops/histograms-and-skewed-data/) shows you examples on how to adjust skewed data.

In [ ]:
# your code here
df['SalePriceAdjusted'] = df['SalePrice'].apply(np.log)


In [ ]:
#Paolo: very good,  I suggest to visualize after you adjust the distribution, it helps to see  the adjusted distribution. 
#It looks more normal
df['SalePriceAdjusted'].hist(bins=50);

## Challenge 2 - Exploring Data with Common Sense

Now that we have a general understanding of the dataset, we start exploring the data with common sense by means of data visualization. Yes, in data analysis and even machine learning you are often required to use common sense. You use your common sense to make a scientific guess (i.e. hypothesis) then use data analytics methods to test your hypothesis.

This dataset is about housing sales. According to common sense, housing prices depend on the following factors:

* **Size of the house** (`GrLivArea`, `LotArea`, and `GarageArea`).

* **Number of rooms** (`BedroomAbvGr`, `KitchenAbvGr`, `FullBath`, `HalfBath`, `BsmtFullBath`, `BsmtHalfBath`).

* **How long the house has been built or remodeled** (`YearBuilt` and `YearRemodAdd`).

* **Neighborhood of the house** (`Neighborhood`).

#### In this challenge, use the appropriate graph type to visualize the relationships between `SalePrice` (or `SalePriceAdjusted`) and the fields above. 

Note that:

* Transform certain columns in order to visualize the data properly based on common sense. For example:
    * Visualizing how the number of half bathrooms affected the sale price probably does not make sense. You can create a new column to calculate the total number of bathrooms/rooms then visualize with the calculated number.
    * `YearBuilt` and `YearRemodAdd` are year numbers not the age of the house. You can create two new columns for how long the house has been built or remodeled then visualize with the calculated columns.
* Make comments to explain your thinking process.

In [ ]:
# your code here

# add cells as needed
#im adding up all the rooms cells, and for baths im diving it /2 since its kind o a half room
df['rooms_total'] = df['BedroomAbvGr'] + df['KitchenAbvGr'] + df['FullBath'] + df['HalfBath']/2 + df['BsmtFullBath'] + df['BsmtHalfBath']/2
df['size_total'] = df['GrLivArea'] + df['LotArea'] + df['GarageArea']
#for house age im taking into consideration only year remodeled, since year remodeled is equal to year built if 
#there has not been any remodeling.
df['age'] = 2020 - df['YearRemodAdd']

#im not using neighbourhood since its a categorical data with multiple values so I think it would need an ANOVA
#analysis to do it.
corr = df[['age', 'size_total', 'rooms_total','SalePriceAdjusted']].corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

sns.heatmap(corr, center=0, mask=mask)


In [ ]:
#Paolo: A couple of suggestions in the cell belows

In [ ]:
#Paolo: for heatmap, if the map is not too busy with many squares it helps to print the values in the square.
#use annot=True
sns.heatmap(corr, center=0, annot=True, mask=mask);

In [ ]:
#Paolo: in general I think you went for the correlation too soon, it helps to support the correlations with preliminary plots
# to understand the data, see below for examples. Also because results you get from correlations matrixes alone can be deceiving, a plot
# gives you more info than a single number.

In [ ]:
## SIZE OF THE HOUSE

# Variables
house_size = ['GrLivArea', 'LotArea', 'GarageArea']

# Plot
f, ax = plt.subplots(1, len(house_size), figsize=(20,5))

for index in range(len(house_size)):
    ax[index].scatter(df[house_size[index]], df["SalePrice"])
    ax[index].set_title(f"{house_size[index]} VS SalePrice", fontweight=700)
    ax[index].set_xlabel(f"{house_size[index]}")
    ax[index].set_ylabel("SalePrice")

In [ ]:
## NUMBER OF ROOMS

# Variables
rooms = ['FullBath', 'BsmtFullBath', 'BedroomAbvGr', 'KitchenAbvGr']
half_rooms = ['HalfBath', 'BsmtHalfBath']

# Total rooms
df['TotalRooms'] = df[rooms].sum(axis=1) + (df[half_rooms] * 0.5).sum(axis=1)
df['TotalBathrooms'] = df[rooms[:2]].sum(axis=1) + (df[half_rooms] * 0.5).sum(axis=1)

# Find median
room_price_mean = df[['SalePrice',"TotalRooms"]].groupby(["TotalRooms"]).median()
bathroom_price_mean = df[['SalePrice',"TotalBathrooms"]].groupby(["TotalBathrooms"]).median()

# Plot

## Variables
rooms_df = [room_price_mean, bathroom_price_mean]
titles = ['#Rooms', '#Bathrooms']
labels = [room_price_mean.index.astype(str), bathroom_price_mean.index.astype(str)]
f, ax = plt.subplots(len(rooms_df), 1, figsize=(10,10))

## Plot
for index in range(len(rooms_df)):
    ax[index].bar(labels[index], rooms_df[index]['SalePrice'])
    ax[index].set_title(f"{titles[index]} VS SalePrice", fontweight=700)
    ax[index].set_xlabel(f"{titles[index]}")
    ax[index].set_ylabel("SalePrice")

In [ ]:
# HOW LONG THE HOUSE HAS BEEN BUILT OR REMODELED

# Variables
current_year = pd.Timestamp.today().year

# Calculate years since built/remodelled
df['YearsSinceBuilt'] =  current_year - df['YearBuilt']
df['YearsSinceRemodel'] = current_year - df['YearRemodAdd']

# Plot
years = ['YearsSinceBuilt', 'YearsSinceRemodel']
f, ax = plt.subplots(1, 2, figsize=(20,5))

for index in range(len(years)):
    ax[index].scatter(df[years[index]], df["SalePrice"])
    ax[index].set_title(f"{years[index]} VS SalePrice", fontweight=700)
    ax[index].set_xlabel(f"{years[index]}")
    ax[index].set_ylabel("SalePrice")

## Bonus Challenge 2 - Exploring Data with Correlation Heatmap

Now you have explored data visualizations with certain fields based on common sense. In the dataset there are many other fields that you are not sure whether they are important factors for the sale price. What is the best way to explore those fields without investigating them individually?

Making scatter matrix is not an option here because there are too many fields which makes it extremely time consuming to create scatter matrix. One option you have is to create a heatmap. Heatmaps are much less expensive to create than scatter matrixes. You can use heatmaps to visualize the pairwise correlations between each two variables.

Here is a [reference](https://seaborn.pydata.org/examples/many_pairwise_correlations.html) you can use to learn how to creat the pairwise correlation heatmap. Your heatmap should look like this [example](https://drive.google.com/file/d/1JhdNvbAnnWDFXEtDoBtx3B2KKIkqsnSH/view?usp=sharing)

In [ ]:
# your code here
sns.set(style="white")
corr2 = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
#Paolo: be careful, the dataset contains many columns 
# I think you are discarding too many columns from your analysis without having checked, for example column 'OverallCond' and others
# I suggest to try the heatmap on the entire dataset and maybe using pairplots to get a quick feeling of the data.


In your heatmap, you can easily identify the highly correlated (either positively or negatively) variables by looking for the grids with darker colors. 

#### In the cell below, summarize what variables are highly correlated to the sale price?

In [ ]:
# your comment here
#negatively correlated: age
#positively correlated: size, rooms, garage, loft front,

## Challenge 3 - Present Your Stories

Now based on your findings from the explorations, summarize and present your stories.

#### Present the top 5 factors that affect the sale price.

Use the following format to present each factor:

1. A title line about the factor.

1. No more than 3 sentences to describe the relationship between the factor and the sale price.

1. Support your point with the appropriate graph.

In [ ]:
# your responses here
# Deciding factors for house pricing

# 1 Factor that definitely impacts in a negative way the price of a house is its building/renovating year (the older 
# it is, the lower the price). A variety of factors affect its price positively (size, qty of rooms, garage, loft front).
# A factor that hasnt been analyzed but surely has an impact on the prize is the neighborhood.
# add cells as needed
sns.heatmap(corr, center=0, mask=mask);
#Paolo: ok!
#Paolo for some reason I find the color scale you choose difficult to read, see below

In [ ]:
#Paolo: I find variations of a single color easier to read, you can experiment with the value of cmap
sns.heatmap(corr, center=0, mask=mask, cmap='Greens');